# Combine multiple ELVIS DEM tiles using `gdalwarp`

In [ ]:
cd ../..

In [ ]:
import glob
import pathlib
import geopandas as gpd

In [ ]:
geojson_path = pathlib.Path("data/validation/elvis_preprocessed.geojson")
data_path = pathlib.Path("data/validation/interim/")
# data_path = pathlib.Path("/gdata1/projects/coastal/intertidal/Elevation_data/Interim")

## Load preprocessed tile list

In [ ]:
preprocessed_tiles_gdf = gpd.read_file(geojson_path)

## Combine each data source/project/year using `gdalwarp`
To compress: `!tar -zcvf elvis_interim.tar.gz $data_path`

In [ ]:
failures = []

for (data_source, project, year), group in preprocessed_tiles_gdf.groupby(
    ["data_source", "project", "year"]
):
    output_name = data_path / f"{data_source.replace('_', '')}_{project}_{year}.tif"

    if not output_name.exists():
        print(output_name)
        try:
            # Run gdalwarp on selected paths
            paths = " ".join(group.vsis3_path)
            !gdalwarp $paths $output_name -t_srs EPSG:3577 -tr 10 10 -tap -dstnodata -9999 -r average -overwrite -multi -wm 80% -co NUM_THREADS=ALL_CPUS -of COG -co COMPRESS=ZSTD -co PREDICTOR=YES -co OVERVIEWS=AUTO -co OVERVIEW_COUNT=4
        except:
            failures.append(output_name)

In [ ]:
failures